In [1]:
import pybaseball as pyb 
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
start_date = '2024-03-01'
end_date = '2025-07-01'
statcast_data = pyb.statcast(start_dt=start_date, end_dt=end_date)

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 355/355 [01:03<00:00,  5.59it/s]


In [3]:
pd.set_option('display.max_columns', None)
statcast_data.sample()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
1570,CH,2024-04-02,85.6,-1.87,5.43,"Varland, Louis",668930,686973,NaN,ball,<NA>,<NA>,<NA>,<NA>,11,Brice Turang singles on a line drive to center...,R,L,R,MIL,MIN,B,<NA>,NaN,0,0,2024,-1.2,0.42,-0.56,3.91,<NA>,656555,686554,1,2,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5.594056,-124.584108,1.807968,-13.583422,22.102756,-28.331876,3.19,1.52,<NA>,<NA>,<NA>,87.3,1564,7.1,746009,543877,467793,666397,650489,621043,670242,621439,596146,53.38,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15,1,Changeup,0,0,0,0,0,0,0,0,Infield shade,Standard,230,0.0,0.03,<NA>,<NA>,<NA>,-0.03,<NA>,0,0,0.59,0.59,26,24,27,25,1,0,<NA>,2,7,1,2.66,1.2,-1.2,35.2,<NA>,<NA>,<NA>,<NA>,<NA>


In [4]:
statcast_data['pitch_type'].unique()

array(['FC', 'SL', 'SI', 'CH', 'FF', 'CU', 'ST', 'KC', 'FS', 'SV', 'PO',
       'KN', 'FA', 'CS', nan, 'EP', 'FO', 'SC'], dtype=object)

In [5]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
from pybaseball import pitching_stats
from pybaseball import batting_stats
from pybaseball import schedule_and_record

In [6]:
playerid_lookup('kershaw', 'clayton')

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,kershaw,clayton,477132,kersc001,kershcl01,2036,2008.0,2025.0


In [7]:
kershaw_stats = statcast_pitcher('2024-03-01', '2025-07-01', 477132)
kershaw_stats.groupby('pitch_type')

Gathering Player Data


In [8]:
# Group MLB pitchers by a number of metrics
major_grouped = statcast_data.groupby(['pitcher', 'pitch_name']).agg(
    pitcher_name = ('player_name', 'max'),
    handedness = ('p_throws', 'max'),
    count_pitch = ('player_name', 'count'),
    spin_rate = ('release_spin_rate', 'mean'), 
    release_speed = ('release_speed', 'mean'),
    horz_break = ('pfx_x', 'mean'),
    vert_break = ('pfx_z', 'mean'),
    extension = ('release_extension', 'mean'),
    spin_axis = ('spin_axis', 'mean')
).reset_index()

# Adjust pfx_x and pfx_z to inches (i think?)
major_grouped['horz_break'] = major_grouped['horz_break'] * 12
major_grouped['vert_break'] = major_grouped['vert_break'] * 12

#Filter for pitches that have been thrown > 30 times
major_grouped = major_grouped[major_grouped['count_pitch'] > 30].drop(columns = ['count_pitch', 'pitcher'])
major_grouped

,pitch_name,pitcher_name,handedness,spin_rate,release_speed,horz_break,vert_break,extension,spin_axis
0,4-Seam Fastball,"Verlander, Justin",R,2406.92228,93.64789,-8.791443,18.967461,6.022206,207.095485
1,Changeup,"Verlander, Justin",R,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218
2,Curveball,"Verlander, Justin",R,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471
4,Slider,"Verlander, Justin",R,2453.971284,87.068074,3.912365,6.154865,6.05473,171.753378
5,Sweeper,"Verlander, Justin",R,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667
...,...,...,...,...,...,...,...,...,...
5911,4-Seam Fastball,"Bloss, Jake",R,2438.634615,93.365385,-4.813846,18.041538,6.899038,204.711538
5913,Curveball,"Bloss, Jake",R,2700.225806,77.958065,9.170323,-15.886452,6.867742,33.290323
5915,Slider,"Bloss, Jake",R,2533.727273,87.610909,5.312727,6.726545,7.009091,160.636364
5963,4-Seam Fastball,"Fermin, José",R,2306.172662,96.568345,-9.966906,17.115971,6.236691,216.453237


In [9]:
pitch_type = ['4-Seam Fastball',
 'Changeup',
 'Slider',
 'Curveball',
 'Cutter',
 'Sinker',
 'Split-Finger',
 'Sweeper']
pitch_type

['4-Seam Fastball',
 'Changeup',
 'Slider',
 'Curveball',
 'Cutter',
 'Sinker',
 'Split-Finger',
 'Sweeper']

In [10]:
features = ['spin_rate', 'release_speed', 'horz_break', 'vert_break', 'extension', 'spin_axis']

In [11]:
# Create pitcher vector dataframe
final_vectors = []
pitcher_names = []

# Loop over each pitcher
for pitcher_name in major_grouped['pitcher_name'].unique():
    pitcher_data = major_grouped[major_grouped['pitcher_name'] == pitcher_name]
    vec = []

    # For each standard pitch type, grab metrics if it exists
    for pt in pitch_type:
        pitch_row = pitcher_data[pitcher_data['pitch_name'] == pt]
        if not pitch_row.empty:
            vals = pitch_row[features].iloc[0].values  # ensure 1D
        else:
            vals = np.zeros(len(features))  # or np.nan
        vec.extend(vals)

    final_vectors.append(vec)
    pitcher_names.append(pitcher_name)

# Step 3: build column names
column_names = []
for pt in pitch_type:
    for f in features:
        column_names.append(f"{pt}_{f}")

# Step 4: create DataFrame
vectors_df = pd.DataFrame(final_vectors, columns=column_names)
vectors_df.insert(0, 'pitcher', pitcher_names)
vectors_df

,pitcher,4-Seam Fastball_spin_rate,4-Seam Fastball_release_speed,4-Seam Fastball_horz_break,4-Seam Fastball_vert_break,4-Seam Fastball_extension,4-Seam Fastball_spin_axis,Changeup_spin_rate,Changeup_release_speed,Changeup_horz_break,Changeup_vert_break,Changeup_extension,Changeup_spin_axis,Slider_spin_rate,Slider_release_speed,Slider_horz_break,Slider_vert_break,Slider_extension,Slider_spin_axis,Curveball_spin_rate,Curveball_release_speed,Curveball_horz_break,Curveball_vert_break,Curveball_extension,Curveball_spin_axis,Cutter_spin_rate,Cutter_release_speed,Cutter_horz_break,Cutter_vert_break,Cutter_extension,Cutter_spin_axis,Sinker_spin_rate,Sinker_release_speed,Sinker_horz_break,Sinker_vert_break,Sinker_extension,Sinker_spin_axis,Split-Finger_spin_rate,Split-Finger_release_speed,Split-Finger_horz_break,Split-Finger_vert_break,Split-Finger_extension,Split-Finger_spin_axis,Sweeper_spin_rate,Sweeper_release_speed,Sweeper_horz_break,Sweeper_vert_break,Sweeper_extension,Sweeper_spin_axis
0,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.839080,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2596.626667,79.198667,12.804800,-2.475200,6.102667,44.866667
1,"Jansen, Kenley",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2430.000000,82.613699,6.816986,-3.516164,6.721918,73.616438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2607.417112,92.308200,4.720107,17.994652,6.806952,176.500891,2283.752137,93.109322,-6.559322,19.381017,6.896610,188.128205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2550.500000,81.313158,14.210526,2.542105,6.678947,61.289474
2,"Chavez, Jesse",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1867.053892,85.014970,-16.016048,1.991856,6.353293,254.479042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2554.764706,75.455294,15.844235,-8.287059,6.318824,64.176471,2197.945671,88.730390,-1.867029,12.495280,6.250085,213.443124,1982.514019,90.611526,-16.032150,7.563738,6.261682,228.570093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Morton, Charlie",2276.329734,94.096761,-12.934983,11.250498,6.154243,239.963455,2068.697778,85.719333,-16.238400,-0.834667,6.270667,267.253333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3100.727811,81.467751,14.284118,-8.933822,6.086331,61.539053,2461.072072,88.411712,0.247568,5.832432,6.169970,209.744745,2167.939394,93.667532,-17.467013,3.863896,6.165584,244.376623,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Scherzer, Max",2323.250000,92.673058,-9.795146,16.077670,6.230825,222.026699,1392.483607,83.171311,-13.856066,1.928852,6.474590,252.131148,2364.162304,85.309744,4.038769,4.104615,6.235897,165.057592,2673.393939,75.123485,12.990000,-8.140909,6.399242,59.098485,2407.980392,86.582353,3.614118,6.581176,6.105882,194.392157,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,"Gibson, Cade",2228.953488,91.923256,9.145116,12.027907,5.995349,139.534884,1952.727273,84.988312,15.565714,1.659740,5.974026,120.012987,2844.604651,85.904651,-2.425116,5.492093,5.913953,194.627907,2957.411215,79.116822,-11.683738,-16.244860,5.855140,322.570093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2256.236559,91.883871,16.802581,8.972903,5.995699,135.20

In [12]:
# Add handedness
mlb_vectors_withhand = vectors_df.merge(major_grouped, left_on = 'pitcher', right_on = 'pitcher_name', how = 'left')
mlb_vectors_withhand = mlb_vectors_withhand.drop(columns = ['pitcher_name', 'pitch_name', 'spin_rate', 'release_speed',	'horz_break', 'vert_break', 'extension', 'spin_axis'])
mlb_vectors_withhand

,pitcher,4-Seam Fastball_spin_rate,4-Seam Fastball_release_speed,4-Seam Fastball_horz_break,4-Seam Fastball_vert_break,4-Seam Fastball_extension,4-Seam Fastball_spin_axis,Changeup_spin_rate,Changeup_release_speed,Changeup_horz_break,Changeup_vert_break,Changeup_extension,Changeup_spin_axis,Slider_spin_rate,Slider_release_speed,Slider_horz_break,Slider_vert_break,Slider_extension,Slider_spin_axis,Curveball_spin_rate,Curveball_release_speed,Curveball_horz_break,Curveball_vert_break,Curveball_extension,Curveball_spin_axis,Cutter_spin_rate,Cutter_release_speed,Cutter_horz_break,Cutter_vert_break,Cutter_extension,Cutter_spin_axis,Sinker_spin_rate,Sinker_release_speed,Sinker_horz_break,Sinker_vert_break,Sinker_extension,Sinker_spin_axis,Split-Finger_spin_rate,Split-Finger_release_speed,Split-Finger_horz_break,Split-Finger_vert_break,Split-Finger_extension,Split-Finger_spin_axis,Sweeper_spin_rate,Sweeper_release_speed,Sweeper_horz_break,Sweeper_vert_break,Sweeper_extension,Sweeper_spin_axis,handedness
0,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
1,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
2,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
3,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
4,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3440,"Bloss, Jake",2438.634615,93.365385,-4.813846,18.041538,6.899038,204.711538,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2533.727273,87.610909,5.312727,6.726545,7.009091,160.636364,2700.225806,77.958065,9.170323,-15.886452,6.867742,33.290323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,R
3441,"Bloss, Jake",2438.634615,93.365385,-4.813846,18.041538,6.899038,204.711538,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2533.727273,87.610909,5.312727,6.726545,7.009091,160.636364,2700.225806,77.958065,9.170323,-15.886452,6.867742,33.290323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,R
3442,"Bloss, Jake",2438.634615,93.365385,-4

In [13]:
trackman = pd.read_csv('../../Files/merged_trackman_games.csv')
trackman

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,Runner1st,Runner2nd,Runner3rd,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,base_state,run_exp,count,delta_run_exp,re24
0,1,2/22/25,05:30.1,1,1,"Seid, Spencer",809262,Left,CSD_TRI,"Dumesnil, Nick",8.250940e+05,Right,CAL_LAN,Undefined,1,Top,0,0,0,Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0.0,0.0,0.0,0,0,90.64925,0.432719,1.900443,2037.144708,156.551992,11:15,4.87015,-2.27294,5.37321,-16.36367,18.60084,-7.55091,3.91243,-1.11980,81.94319,-2.683311,0.559518,0.425584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.33244,9.52567,2.10803,50.0,4.89779,-4.00496,-131.38274,0.48208,7.51301,29.88203,-15.65521,CSD_TRI,CAL_LAN,UCSanDiego,D1,BW,20250222-UCSanDiego-1,a863ac20-f18a-11ef-9456-e7fd19fa9ca5,88.56641,NaN,NaN,8.70606,NaN,NaN,NaN,NaN,NaN,NaN,f92401d5-ad19-4ab4-801b-f579f4bca033,2/23/25,05:30.1,2025-02-22T18:05:30.0717056-08:00,2025-02-23T02:05:30.0717056Z,NaN,v3,NaN,471762.0,NaN,"Gonzalez, Emiliano",1000081438,Right,CSD_TRI,161a4951-5cb1-4e14-bccc-efd6a6b7a50f,55.12203,-132.54258,14.94101,4.86729,1.08972,-7.82761,2.26914,-4.29657,3.75651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,000/0,0.72,0-0,NaN,-0.0
1,2,2/22/25,05:45.5,1,2,"Seid, Spencer",809262,Left,CSD_TRI,"Dumesnil, Nick",8.250940e+05,Right,CAL_LAN,Undefined,1,Top,0,1,0,Fastball,Sinker,BallCalled,Undefined,Undefined,Undefined,0.0,0.0,0.0,0,0,90.36915,-2.298232,5.488243,1942.903753,126.818131,10:15,4.73413,-2.12090,5.66248,-21.98430,13.10355,-15.85166,0.74802,1.69154,81.89485,-6.411002,2.672846,0.426334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.78292,7.70910,1.66607,50.0,4.52514,-11.85353,-130.49855,-5.99917,16.72644,29.64637,-18.99334,CSD_TRI,CAL_LAN

In [14]:
# Adjust pitch name types to match statcast
def clean_pitch_names(row):
    
    if row['TaggedPitchType'] == 'Splitter':
        row['TaggedPitchType'] = 'Split-Finger'
    if row['TaggedPitchType'] == 'FourSeamFastBall':
        row['TaggedPitchType'] = 'Fastball'
    if row['TaggedPitchType'] == 'ChangeUp':
        row["TaggedPitchType"] = 'Changeup'
    if row['TaggedPitchType'] == 'Fastball':
        row['TaggedPitchType'] = '4-Seam Fastball'
    
    row['HorzBreak'] = row['HorzBreak'] * (-1)
        
    return row

trackman = trackman.apply(clean_pitch_names, axis = 1)
trackman

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,BatterId,BatterSide,BatterTeam,PitcherSet,Inning,Top/Bottom,Outs,Balls,Strikes,TaggedPitchType,AutoPitchType,PitchCall,KorBB,TaggedHitType,PlayResult,Runner1st,Runner2nd,Runner3rd,OutsOnPlay,RunsScored,RelSpeed,VertRelAngle,HorzRelAngle,SpinRate,SpinAxis,Tilt,RelHeight,RelSide,Extension,VertBreak,InducedVertBreak,HorzBreak,PlateLocHeight,PlateLocSide,ZoneSpeed,VertApprAngle,HorzApprAngle,ZoneTime,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,PositionAt110Z,Distance,LastTrackedDistance,Bearing,HangTime,pfxx,pfxz,x0,y0,z0,vx0,vy0,vz0,ax0,ay0,az0,HomeTeam,AwayTeam,Stadium,Level,League,GameID,PitchUID,EffectiveVelo,MaxHeight,MeasuredDuration,SpeedDrop,PitchLastMeasuredX,PitchLastMeasuredY,PitchLastMeasuredZ,ContactPositionX,ContactPositionY,ContactPositionZ,GameUID,UTCDate,UTCTime,LocalDateTime,UTCDateTime,AutoHitType,System,HomeTeamForeignID,AwayTeamForeignID,GameForeignID,Catcher,CatcherId,CatcherThrows,CatcherTeam,PlayID,PitchTrajectoryXc0,PitchTrajectoryXc1,PitchTrajectoryXc2,PitchTrajectoryYc0,PitchTrajectoryYc1,PitchTrajectoryYc2,PitchTrajectoryZc0,PitchTrajectoryZc1,PitchTrajectoryZc2,HitSpinAxis,HitTrajectoryXc0,HitTrajectoryXc1,HitTrajectoryXc2,HitTrajectoryXc3,HitTrajectoryXc4,HitTrajectoryXc5,HitTrajectoryXc6,HitTrajectoryXc7,HitTrajectoryXc8,HitTrajectoryYc0,HitTrajectoryYc1,HitTrajectoryYc2,HitTrajectoryYc3,HitTrajectoryYc4,HitTrajectoryYc5,HitTrajectoryYc6,HitTrajectoryYc7,HitTrajectoryYc8,HitTrajectoryZc0,HitTrajectoryZc1,HitTrajectoryZc2,HitTrajectoryZc3,HitTrajectoryZc4,HitTrajectoryZc5,HitTrajectoryZc6,HitTrajectoryZc7,HitTrajectoryZc8,ThrowSpeed,PopTime,ExchangeTime,TimeToBase,CatchPositionX,CatchPositionY,CatchPositionZ,ThrowPositionX,ThrowPositionY,ThrowPositionZ,BasePositionX,BasePositionY,BasePositionZ,ThrowTrajectoryXc0,ThrowTrajectoryXc1,ThrowTrajectoryXc2,ThrowTrajectoryYc0,ThrowTrajectoryYc1,ThrowTrajectoryYc2,ThrowTrajectoryZc0,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence,base_state,run_exp,count,delta_run_exp,re24
0,1,2/22/25,05:30.1,1,1,"Seid, Spencer",809262,Left,CSD_TRI,"Dumesnil, Nick",8.250940e+05,Right,CAL_LAN,Undefined,1,Top,0,0,0,4-Seam Fastball,Four-Seam,BallCalled,Undefined,Undefined,Undefined,0.0,0.0,0.0,0,0,90.64925,0.432719,1.900443,2037.144708,156.551992,11:15,4.87015,-2.27294,5.37321,-16.36367,18.60084,7.55091,3.91243,-1.11980,81.94319,-2.683311,0.559518,0.425584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.33244,9.52567,2.10803,50.0,4.89779,-4.00496,-131.38274,0.48208,7.51301,29.88203,-15.65521,CSD_TRI,CAL_LAN,UCSanDiego,D1,BW,20250222-UCSanDiego-1,a863ac20-f18a-11ef-9456-e7fd19fa9ca5,88.56641,NaN,NaN,8.70606,NaN,NaN,NaN,NaN,NaN,NaN,f92401d5-ad19-4ab4-801b-f579f4bca033,2/23/25,05:30.1,2025-02-22T18:05:30.0717056-08:00,2025-02-23T02:05:30.0717056Z,NaN,v3,NaN,471762.0,NaN,"Gonzalez, Emiliano",1000081438,Right,CSD_TRI,161a4951-5cb1-4e14-bccc-efd6a6b7a50f,55.12203,-132.54258,14.94101,4.86729,1.08972,-7.82761,2.26914,-4.29657,3.75651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,High,High,NaN,NaN,NaN,NaN,NaN,000/0,0.72,0-0,NaN,-0.0
1,2,2/22/25,05:45.5,1,2,"Seid, Spencer",809262,Left,CSD_TRI,"Dumesnil, Nick",8.250940e+05,Right,CAL_LAN,Undefined,1,Top,0,1,0,4-Seam Fastball,Sinker,BallCalled,Undefined,Undefined,Undefined,0.0,0.0,0.0,0,0,90.36915,-2.298232,5.488243,1942.903753,126.818131,10:15,4.73413,-2.12090,5.66248,-21.98430,13.10355,15.85166,0.74802,1.69154,81.89485,-6.411002,2.672846,0.426334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.78292,7.70910,1.66607,50.0,4.52514,-11.85353,-130.49855,-5.99917,16.72644,29.64637,-18.99334,CSD

In [15]:
# Group by ucsd pitcher and various metrics
ucsd = trackman[trackman['PitcherTeam'] == "CSD_TRI"].groupby(['Pitcher', 'TaggedPitchType']).agg(
    handedness = ('PitcherThrows', 'max'),
    count_pitch = ('TaggedPitchType', 'count'),
    release_speed = ('RelSpeed', 'mean'),
    spin_rate = ('SpinRate', 'mean'),
    spin_axis = ('SpinAxis', 'mean'),
    extension = ('Extension', 'mean'),
    vert_break = ('InducedVertBreak', 'mean'),
    horz_break = ('HorzBreak', 'mean')
).reset_index()

ucsd = ucsd[ucsd['count_pitch'] > 8]
ucsd

,Pitcher,TaggedPitchType,handedness,count_pitch,release_speed,spin_rate,spin_axis,extension,vert_break,horz_break
0,"Cazares, Julian",4-Seam Fastball,Left,66,88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465
1,"Cazares, Julian",Changeup,Left,11,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775
4,"Cazares, Julian",Slider,Left,42,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663
9,"Dalquist, Matthew",4-Seam Fastball,Right,390,88.917354,2428.225993,202.826777,5.742637,20.117488,-8.213400
10,"Dalquist, Matthew",Changeup,Right,91,79.465425,1781.151688,253.207777,5.812097,5.499094,-15.234964
...,...,...,...,...,...,...,...,...,...,...
94,"Villar, Jake",Sinker,Left,47,85.483072,2168.623190,123.516458,5.156891,10.306133,13.589755
95,"Villar, Jake",Slider,Left,142,78.654628,2188.153356,277.418471,4.770237,-0.179325,-8.444508
96,"Villar, Jake",Sweeper,Left,24,78.741176,2277.967680,276.632578,5.070667,0.548352,-10.121163
97,"Weber, Chapman",4-Seam Fastball,Right,236,89.697219,2265.244352,203.660116,5.122914,23.956657,-10.018547


In [16]:
pitch_type_ucsd = list(ucsd['TaggedPitchType'].unique())
pitch_type_ucsd

['4-Seam Fastball',
 'Changeup',
 'Slider',
 'Curveball',
 'Cutter',
 'Sinker',
 'Split-Finger',
 'Sweeper']

In [17]:
features_ucsd = ['release_speed', 'spin_rate', 'spin_axis', 'extension', 'vert_break', 'horz_break']

In [18]:
final_vectors = []
pitcher_names = []

# Loop over each pitcher
for pitcher_name in ucsd['Pitcher'].unique():
    pitcher_data = ucsd[ucsd['Pitcher'] == pitcher_name]
    vec = []

    # For each standard pitch type, grab metrics if it exists
    for pt in pitch_type_ucsd:
        pitch_row = pitcher_data[pitcher_data['TaggedPitchType'] == pt]
        if not pitch_row.empty:
            vals = pitch_row[features_ucsd].iloc[0].values  # ensure 1D
        else:
            vals = np.zeros(len(features))  # or np.nan
        vec.extend(vals)

    final_vectors.append(vec)
    pitcher_names.append(pitcher_name)

# Step 3: build column names
column_names = []
for pt in pitch_type_ucsd:
    for f in features_ucsd:
        column_names.append(f"{pt}_{f}")

# Step 4: create DataFrame
ucsd_vectors = pd.DataFrame(final_vectors, columns=column_names)
ucsd_vectors.insert(0, 'pitcher', pitcher_names)
ucsd_vectors

,pitcher,4-Seam Fastball_release_speed,4-Seam Fastball_spin_rate,4-Seam Fastball_spin_axis,4-Seam Fastball_extension,4-Seam Fastball_vert_break,4-Seam Fastball_horz_break,Changeup_release_speed,Changeup_spin_rate,Changeup_spin_axis,Changeup_extension,Changeup_vert_break,Changeup_horz_break,Slider_release_speed,Slider_spin_rate,Slider_spin_axis,Slider_extension,Slider_vert_break,Slider_horz_break,Curveball_release_speed,Curveball_spin_rate,Curveball_spin_axis,Curveball_extension,Curveball_vert_break,Curveball_horz_break,Cutter_release_speed,Cutter_spin_rate,Cutter_spin_axis,Cutter_extension,Cutter_vert_break,Cutter_horz_break,Sinker_release_speed,Sinker_spin_rate,Sinker_spin_axis,Sinker_extension,Sinker_vert_break,Sinker_horz_break,Split-Finger_release_speed,Split-Finger_spin_rate,Split-Finger_spin_axis,Split-Finger_extension,Split-Finger_vert_break,Split-Finger_horz_break,Sweeper_release_speed,Sweeper_spin_rate,Sweeper_spin_axis,Sweeper_extension,Sweeper_vert_break,Sweeper_horz_break
0,"Cazares, Julian",88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,"Dalquist, Matthew",88.917354,2428.225993,202.826777,5.742637,20.117488,-8.213400,79.465425,1781.151688,253.207777,5.812097,5.499094,-15.234964,82.720486,2447.634032,113.902140,5.161007,3.741759,5.824260,74.926023,2459.541861,44.089341,5.139454,-10.852446,12.105688,83.199194,2488.615077,125.433412,5.413208,5.026725,5.189934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,"Davidson, Garrett",85.661506,2146.994880,154.427683,4.903740,15.957478,7.336815,77.308087,1515.452132,95.027869,4.959643,2.559521,14.410717,74.155766,2276.647706,280.535485,4.491794,-5.350577,-9.668084,71.310709,2357.390834,291.703567,4.155051,-8.962203,-14.695104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,"Ernisse, Zach",88.736867,2188.778474,164.486918,5.927394,23.291826,6.075582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.042626,2379.753928,276.029631,5.293233,0.234685,-6.669285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
4,"Gregson, Niccolas",89.617333,2255.416915,197.737886,6.341022,18.939651,-5.994221,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,82.411034,2528.056539,115.903588,5.903371,3.448766,2.824657,74.969531,2815.166014,32.382876,5.745636,-15.135522,8.280808,84.007924,2394.278370,179.781303,6.390396,8.141304,-0.327554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
5,"Hasegawa, Sam",89.421907,2416.848309,184.654447,5.663910,10.826143,-0.400802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.452536,2502.029758,283.178014,5.508476,-1.787087,-9.056739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.132758,2406.797889,187.684837,6.128570,11.572052,-1.267020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
6,"Huy, Nathan",85.530805,1922.655264,180.828451,5.413591,12.213211,0.511295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,75.788416,2253.

In [19]:
ucsd_vectors_withhand = ucsd_vectors.merge(ucsd, left_on = 'pitcher', right_on = 'Pitcher', how = 'left')
ucsd_vectors_withhand = ucsd_vectors_withhand.drop(columns = ['TaggedPitchType', 'Pitcher', 'count_pitch', 'release_speed', 'spin_rate', 'spin_axis', 'extension', 'vert_break', 'horz_break'])
ucsd_vectors_withhand

,pitcher,4-Seam Fastball_release_speed,4-Seam Fastball_spin_rate,4-Seam Fastball_spin_axis,4-Seam Fastball_extension,4-Seam Fastball_vert_break,4-Seam Fastball_horz_break,Changeup_release_speed,Changeup_spin_rate,Changeup_spin_axis,Changeup_extension,Changeup_vert_break,Changeup_horz_break,Slider_release_speed,Slider_spin_rate,Slider_spin_axis,Slider_extension,Slider_vert_break,Slider_horz_break,Curveball_release_speed,Curveball_spin_rate,Curveball_spin_axis,Curveball_extension,Curveball_vert_break,Curveball_horz_break,Cutter_release_speed,Cutter_spin_rate,Cutter_spin_axis,Cutter_extension,Cutter_vert_break,Cutter_horz_break,Sinker_release_speed,Sinker_spin_rate,Sinker_spin_axis,Sinker_extension,Sinker_vert_break,Sinker_horz_break,Split-Finger_release_speed,Split-Finger_spin_rate,Split-Finger_spin_axis,Split-Finger_extension,Split-Finger_vert_break,Split-Finger_horz_break,Sweeper_release_speed,Sweeper_spin_rate,Sweeper_spin_axis,Sweeper_extension,Sweeper_vert_break,Sweeper_horz_break,handedness
0,"Cazares, Julian",88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,Left
1,"Cazares, Julian",88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,Left
2,"Cazares, Julian",88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,Left
3,"Dalquist, Matthew",88.917354,2428.225993,202.826777,5.742637,20.117488,-8.213400,79.465425,1781.151688,253.207777,5.812097,5.499094,-15.234964,82.720486,2447.634032,113.902140,5.161007,3.741759,5.824260,74.926023,2459.541861,44.089341,5.139454,-10.852446,12.105688,83.199194,2488.615077,125.433412,5.413208,5.026725,5.189934,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,Right
4,"Dalquist, Matthew",88.917354,2428.225993,202.826777,5.742637,20.117488,-8.213400,79.465425,1781.151688,253.207777,5.812097,5.499094,-15.234964,82.720486,2447.634032,113.902140,5.161007,3.741759,5.824260,74.926023,2459.541861,44.089341,5.139454,-10.852446,12.105688,83.199194,2488.615077,125.433412,5.413208,5.026725,5.189934,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,Right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,"Villar, Jake",85.682666,2159.920294,147.640075,5.568090,12.602043,6.992488,77.891877,1584.741558,128.733564,5.510417,8.661711,10.245849,78.654628,2188.153356,277.418471,4.770237,-0.179325,-8.444508,77.251492,1995.569223,289.249605,4.506115,-7.865252,-9.168466,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,85.483072,2168.62319,123.516458,5.156891,10.306133,13.589755,0.0,0.0,0.0,0.0,0.0,0.0,78.741176,2277.96768,276.632578,5.070667,0.548352,-10.121163,Left
57,"Villar, Jake",85.682666,2159.920294,147.640075,

In [20]:
mlb_vectors_withhand

,pitcher,4-Seam Fastball_spin_rate,4-Seam Fastball_release_speed,4-Seam Fastball_horz_break,4-Seam Fastball_vert_break,4-Seam Fastball_extension,4-Seam Fastball_spin_axis,Changeup_spin_rate,Changeup_release_speed,Changeup_horz_break,Changeup_vert_break,Changeup_extension,Changeup_spin_axis,Slider_spin_rate,Slider_release_speed,Slider_horz_break,Slider_vert_break,Slider_extension,Slider_spin_axis,Curveball_spin_rate,Curveball_release_speed,Curveball_horz_break,Curveball_vert_break,Curveball_extension,Curveball_spin_axis,Cutter_spin_rate,Cutter_release_speed,Cutter_horz_break,Cutter_vert_break,Cutter_extension,Cutter_spin_axis,Sinker_spin_rate,Sinker_release_speed,Sinker_horz_break,Sinker_vert_break,Sinker_extension,Sinker_spin_axis,Split-Finger_spin_rate,Split-Finger_release_speed,Split-Finger_horz_break,Split-Finger_vert_break,Split-Finger_extension,Split-Finger_spin_axis,Sweeper_spin_rate,Sweeper_release_speed,Sweeper_horz_break,Sweeper_vert_break,Sweeper_extension,Sweeper_spin_axis,handedness
0,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
1,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
2,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
3,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
4,"Verlander, Justin",2406.922280,93.647890,-8.791443,18.967461,6.022206,207.095485,1758.83908,84.308429,-13.354023,9.471264,6.048659,229.563218,2453.971284,87.068074,3.912365,6.154865,6.054730,171.753378,2692.281947,77.870791,7.358702,-12.787181,5.997972,38.176471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2596.626667,79.198667,12.8048,-2.4752,6.102667,44.866667,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3440,"Bloss, Jake",2438.634615,93.365385,-4.813846,18.041538,6.899038,204.711538,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2533.727273,87.610909,5.312727,6.726545,7.009091,160.636364,2700.225806,77.958065,9.170323,-15.886452,6.867742,33.290323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,R
3441,"Bloss, Jake",2438.634615,93.365385,-4.813846,18.041538,6.899038,204.711538,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2533.727273,87.610909,5.312727,6.726545,7.009091,160.636364,2700.225806,77.958065,9.170323,-15.886452,6.867742,33.290323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,R
3442,"Bloss, Jake",2438.634615,93.365385,-4

In [21]:
# Encode handedness for KNN model (Left = 0, Right = 1)
def encode_handedness(row):
    
    if (row['handedness'] == 'R') | (row['handedness'] == 'Right'):
        row['handedness'] = 1
    if (row['handedness'] == 'L') | (row['handedness'] == 'Left'):
        row['handedness'] = 0
        
    return row

mlb_vectors_withhand = mlb_vectors_withhand.apply(encode_handedness, axis = 1).copy().drop_duplicates(subset = 'pitcher')
ucsd_vectors_withhand = ucsd_vectors_withhand.apply(encode_handedness, axis = 1).copy().drop_duplicates(subset = 'pitcher')
display(ucsd_vectors_withhand)

#Spoiler, steele is compared to bello
display(mlb_vectors_withhand[mlb_vectors_withhand['pitcher'] == 'Bello, Brayan'])

,pitcher,4-Seam Fastball_release_speed,4-Seam Fastball_spin_rate,4-Seam Fastball_spin_axis,4-Seam Fastball_extension,4-Seam Fastball_vert_break,4-Seam Fastball_horz_break,Changeup_release_speed,Changeup_spin_rate,Changeup_spin_axis,Changeup_extension,Changeup_vert_break,Changeup_horz_break,Slider_release_speed,Slider_spin_rate,Slider_spin_axis,Slider_extension,Slider_vert_break,Slider_horz_break,Curveball_release_speed,Curveball_spin_rate,Curveball_spin_axis,Curveball_extension,Curveball_vert_break,Curveball_horz_break,Cutter_release_speed,Cutter_spin_rate,Cutter_spin_axis,Cutter_extension,Cutter_vert_break,Cutter_horz_break,Sinker_release_speed,Sinker_spin_rate,Sinker_spin_axis,Sinker_extension,Sinker_vert_break,Sinker_horz_break,Split-Finger_release_speed,Split-Finger_spin_rate,Split-Finger_spin_axis,Split-Finger_extension,Split-Finger_vert_break,Split-Finger_horz_break,Sweeper_release_speed,Sweeper_spin_rate,Sweeper_spin_axis,Sweeper_extension,Sweeper_vert_break,Sweeper_horz_break,handedness
0,"Cazares, Julian",88.471789,2151.435786,164.676380,5.843351,18.680750,4.907465,79.701473,1754.774606,95.453819,4.922805,3.857876,17.701775,79.361717,2349.437108,267.437374,5.162385,-0.939638,-3.670663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0
3,"Dalquist, Matthew",88.917354,2428.225993,202.826777,5.742637,20.117488,-8.213400,79.465425,1781.151688,253.207777,5.812097,5.499094,-15.234964,82.720486,2447.634032,113.902140,5.161007,3.741759,5.824260,74.926023,2459.541861,44.089341,5.139454,-10.852446,12.105688,83.199194,2488.615077,125.433412,5.413208,5.026725,5.189934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1
8,"Davidson, Garrett",85.661506,2146.994880,154.427683,4.903740,15.957478,7.336815,77.308087,1515.452132,95.027869,4.959643,2.559521,14.410717,74.155766,2276.647706,280.535485,4.491794,-5.350577,-9.668084,71.310709,2357.390834,291.703567,4.155051,-8.962203,-14.695104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0
12,"Ernisse, Zach",88.736867,2188.778474,164.486918,5.927394,23.291826,6.075582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.042626,2379.753928,276.029631,5.293233,0.234685,-6.669285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0
14,"Gregson, Niccolas",89.617333,2255.416915,197.737886,6.341022,18.939651,-5.994221,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,82.411034,2528.056539,115.903588,5.903371,3.448766,2.824657,74.969531,2815.166014,32.382876,5.745636,-15.135522,8.280808,84.007924,2394.278370,179.781303,6.390396,8.141304,-0.327554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1
18,"Hasegawa, Sam",89.421907,2416.848309,184.654447,5.663910,10.826143,-0.400802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.452536,2502.029758,283.178014,5.508476,-1.787087,-9.056739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,89.132758,2406.797889,187.684837,6.128570,11.572052,-1.267020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0
21,"Huy, Nathan",85.530805,1922.655264,180.828451,5.413591,12.213211,0.511295,0.000000,0.000000,0.000000,0.000000,0.0000

,pitcher,4-Seam Fastball_spin_rate,4-Seam Fastball_release_speed,4-Seam Fastball_horz_break,4-Seam Fastball_vert_break,4-Seam Fastball_extension,4-Seam Fastball_spin_axis,Changeup_spin_rate,Changeup_release_speed,Changeup_horz_break,Changeup_vert_break,Changeup_extension,Changeup_spin_axis,Slider_spin_rate,Slider_release_speed,Slider_horz_break,Slider_vert_break,Slider_extension,Slider_spin_axis,Curveball_spin_rate,Curveball_release_speed,Curveball_horz_break,Curveball_vert_break,Curveball_extension,Curveball_spin_axis,Cutter_spin_rate,Cutter_release_speed,Cutter_horz_break,Cutter_vert_break,Cutter_extension,Cutter_spin_axis,Sinker_spin_rate,Sinker_release_speed,Sinker_horz_break,Sinker_vert_break,Sinker_extension,Sinker_spin_axis,Split-Finger_spin_rate,Split-Finger_release_speed,Split-Finger_horz_break,Split-Finger_vert_break,Split-Finger_extension,Split-Finger_spin_axis,Sweeper_spin_rate,Sweeper_release_speed,Sweeper_horz_break,Sweeper_vert_break,Sweeper_extension,Sweeper_spin_axis,handedness
2634,"Bello, Brayan",2180.79257,95.210526,-8.273355,13.035789,6.354799,226.003096,1806.034627,87.918888,-16.057764,3.242644,6.418783,249.002099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2515.630631,88.582883,0.787027,2.784865,6.382883,166.207207,2072.633415,95.504162,-17.098338,4.277552,6.358078,233.366585,0.0,0.0,0.0,0.0,0.0,0.0,2395.497137,86.498187,8.588621,-1.22187,6.251289,41.419847,1


In [22]:
from sklearn.preprocessing import StandardScaler

x_ucsd = ucsd_vectors_withhand.drop(columns = ['pitcher']).copy().drop_duplicates(subset = '4-Seam Fastball_release_speed')
x_mlb = mlb_vectors_withhand.drop(columns = ['pitcher']).copy().drop_duplicates(subset = '4-Seam Fastball_release_speed')

scaler = StandardScaler()
x_all = pd.concat([x_ucsd, x_mlb], axis = 0)
x_all_scaled = scaler.fit_transform(x_all)

x_mlb_scaled = x_all_scaled[:len(x_mlb)]
x_ucsd_scaled = x_all_scaled[len(x_mlb):]

In [23]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors = 10, metric = 'euclidean')
knn.fit(x_mlb_scaled)

distances, indices = knn.kneighbors(x_ucsd_scaled)

results = []

for i, (dists, idxs) in enumerate(zip(distances, indices)):
    team_pitcher = ucsd_vectors_withhand.iloc[i]['pitcher']
    for rank, (dist, idx) in enumerate(zip(dists, idxs), start=1):
        mlb_pitcher = mlb_vectors_withhand.iloc[idx]['pitcher']
        results.append({
            'Team Pitcher': team_pitcher,
            'Similar MLB Pitcher': mlb_pitcher,
            'Similarity Rank': rank,
            'Distance': dist
        })

top10_df = pd.DataFrame(results)
top10_df[top10_df['Similarity Rank'] == 1]

,Team Pitcher,Similar MLB Pitcher,Similarity Rank,Distance
0,"Cazares, Julian","Smith, Cade",1,2.350211
10,"Dalquist, Matthew","Thompson, Keegan",1,2.825474
20,"Davidson, Garrett","Hill, Jaden",1,1.565082
30,"Ernisse, Zach","Mattson, Isaac",1,2.405401
40,"Gregson, Niccolas","Herget, Kevin",1,1.279899
50,"Hasegawa, Sam","Montgomery, Mason",1,3.098777
60,"Huy, Nathan","Rasmussen, Drew",1,1.932304
70,"King, Devon","Hahn, Jesse",1,1.215358
80,"Marchetti, Landon","Cruz, Fernando",1,3.013424
90,"Murdock, Steele","Fernandez, Ryan",1,2.072634


In [80]:
def find_similar_pitchers(
    team_df: pd.DataFrame,
    mlb_df: pd.DataFrame,
    team_vectors_df: pd.DataFrame,
    mlb_vectors_df: pd.DataFrame,
    team_pitcher_name: str,
    n_neighbors: int = 3
) -> pd.DataFrame:
    
    
   # Get rid of duplicates in team_df
    
   # --- 1. Get pitch types for the team pitcher
    team_pitch_types = set(team_df[team_df['Pitcher'] == team_pitcher_name]['TaggedPitchType'].unique())

    # --- 2. Map MLB pitchers to their pitch types
    mlb_pitch_types_by_pitcher = mlb_df.groupby('pitcher_name')['pitch_name'].apply(set).to_dict()

    # --- 3. Filter MLB pitchers who throw at least all the same pitch types
    compatible_mlb_pitchers = [
        p for p, pitches in mlb_pitch_types_by_pitcher.items()
        if team_pitch_types.issubset(pitches)
    ]

    # --- 4. Subset MLB vectors to compatible pitchers
    mlb_filtered_vectors = mlb_vectors_df[mlb_vectors_df['pitcher'].isin(compatible_mlb_pitchers)]
    if mlb_filtered_vectors.empty:
        raise ValueError(f"No MLB pitchers found with compatible pitch types for {team_pitcher_name}.")
    

    # --- 5. Get team vector
    team_vector_row = team_vectors_df[team_vectors_df['pitcher'] == team_pitcher_name]
    if team_vector_row.empty:
        raise ValueError(f"Pitcher '{team_pitcher_name}' not found in team_vectors_df.")
    team_vector = team_vector_row.drop(columns='pitcher').values
    
    
    # ---- Filter for handedness
    handedness = team_vector_row['handedness'].iloc[0]
    mlb_filtered_hand = mlb_filtered_vectors[mlb_filtered_vectors['handedness'] == handedness]
    if mlb_filtered_hand.empty:
        raise ValueError(f"No MLB pitchers found with compatible pitch types for {team_pitcher_name}.")

    # --- 6. Scale both MLB and team vectors
    X_mlb = mlb_filtered_hand.drop(columns='pitcher').values
    X_all = np.vstack([X_mlb, team_vector])
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_all)
    X_mlb_scaled = X_scaled[:-1]
    X_team_scaled = X_scaled[-1].reshape(1, -1)

    # --- 7. Fit KNN
    knn = NearestNeighbors(n_neighbors=n_neighbors)
    knn.fit(X_mlb_scaled)
    distances, indices = knn.kneighbors(X_team_scaled)

    # --- 8. Return results
    similar_pitchers = mlb_filtered_hand.iloc[indices[0]].copy()
    similar_pitchers['distance'] = distances[0]
    
    return similar_pitchers.reset_index(drop=True)




similar_pitches_df = find_similar_pitchers(
    team_df = ucsd,
    mlb_df = major_grouped,
    team_vectors_df=  ucsd_vectors_withhand,
    mlb_vectors_df = mlb_vectors_withhand,
    team_pitcher_name='Villar, Jake',
    n_neighbors = 7
)

display(similar_pitches_df)

,pitcher,4-Seam Fastball_spin_rate,4-Seam Fastball_release_speed,4-Seam Fastball_horz_break,4-Seam Fastball_vert_break,4-Seam Fastball_extension,4-Seam Fastball_spin_axis,Changeup_spin_rate,Changeup_release_speed,Changeup_horz_break,Changeup_vert_break,Changeup_extension,Changeup_spin_axis,Slider_spin_rate,Slider_release_speed,Slider_horz_break,Slider_vert_break,Slider_extension,Slider_spin_axis,Curveball_spin_rate,Curveball_release_speed,Curveball_horz_break,Curveball_vert_break,Curveball_extension,Curveball_spin_axis,Cutter_spin_rate,Cutter_release_speed,Cutter_horz_break,Cutter_vert_break,Cutter_extension,Cutter_spin_axis,Sinker_spin_rate,Sinker_release_speed,Sinker_horz_break,Sinker_vert_break,Sinker_extension,Sinker_spin_axis,Split-Finger_spin_rate,Split-Finger_release_speed,Split-Finger_horz_break,Split-Finger_vert_break,Split-Finger_extension,Split-Finger_spin_axis,Sweeper_spin_rate,Sweeper_release_speed,Sweeper_horz_break,Sweeper_vert_break,Sweeper_extension,Sweeper_spin_axis,handedness,distance
0,"Sears, JP",2061.068209,92.064828,11.672028,14.460356,6.398814,132.313167,1608.923077,83.710134,16.456410,2.948132,6.384737,105.175824,2350.305638,79.620772,-5.694125,8.474421,6.373591,263.893175,2366.729730,79.425225,-4.708108,-1.940541,6.414414,286.819820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1884.114914,90.098778,16.016333,6.438337,6.438142,121.088020,0.0,0.0,0.0,0.0,0.0,0.0,2375.261014,78.765503,-13.196808,5.054663,6.379052,272.917706,0,16.027932
1,"Gibson, Cade",2228.953488,91.923256,9.145116,12.027907,5.995349,139.534884,1952.727273,84.988312,15.565714,1.659740,5.974026,120.012987,2844.604651,85.904651,-2.425116,5.492093,5.913953,194.627907,2957.411215,79.116822,-11.683738,-16.244860,5.855140,322.570093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2256.236559,91.883871,16.802581,8.972903,5.995699,135.204301,0.0,0.0,0.0,0.0,0.0,0.0,2931.767442,80.218605,-12.831628,-0.666977,5.904651,309.069767,0,17.137331
2,"Wicks, Jordan",2316.444444,92.598489,4.174610,17.591335,6.297733,153.593434,1511.473214,82.226786,11.638393,6.844821,6.360714,125.183036,2605.098039,85.954902,-3.362353,1.538824,6.452941,244.176471,2797.019231,78.401923,-7.227692,-11.693077,6.476923,321.557692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2155.508475,91.794068,14.303390,11.600339,6.274576,140.466102,0.0,0.0,0.0,0.0,0.0,0.0,2759.543478,82.576087,-13.899130,-0.516522,6.415217,311.369565,0,17.239029
3,"Sandoval, Patrick",2034.370079,93.637402,6.799843,16.144724,6.164567,151.940945,1699.338863,82.905687,14.129005,9.927014,6.307583,115.464455,2543.477212,87.772386,-7.533941,3.873137,6.171314,241.434316,2752.702290,76.316794,-10.801832,-13.374962,6.249618,325.045802,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1963.241636,93.702230,12.981413,13.518959,6.194796,153.360595,0.0,0.0,0.0,0.0,0.0,0.0,2780.027778,81.336111,-17.430000,-1.268333,6.229167,303.305556,0,17.292673
4,"Wrobleski, Justin",2381.212302,95.265675,7.683810,16.285000,5.381944,142.996032,1547.779661,87.042373,11.646102,4.903729,5.423729,119.898305,2376.279570,87.702688,-2.922581,1.441290,5.339785,245.000000,2382.136752,81.756410,-6.739487,-8.593846,5.158974,320.410256,2455.382716,92.092593,-1.136296,8.330370,5.524691,160.135802,2150.550000,94.920714,14.421429,10.528286,5.577143,142.800000,0.0,0.0,0.0,0.0,0.0,0.0,2372.100000,84.464000,-13.051200,-0.794400,5.076000,302.720000,0,17.536608
5,"Povich, Cade",2337.684422,92.341005,9.649327,17.098131,6.289648,140.160804,1342.006645,84.956146,11.655150,6.991894,6.323920,123.980066,2381.356322,83.368966,-5.357241,2.951724,6.327586,269.701149,2500.159004,77.365900,-10.399540,-14.292644,6.211686,322.459770,2485.288462,88.294231,-3.062308,9.304615,6.286538,176.769231,2155.230769,91.647692,15.791077,11.757231,6.294231,141.384615,0.0,0.0,0.0,0.0,0.0,0.0,2398.945055,82.374359,-10.259341,1.246593,6.162271,299.743590,0,17.789130
6,"Fried, Max",2226.544369,94.116382,0.422867,11.821843,6.554266,161.523038,1526.752197,85.34